In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import log_loss
import lightgbm as lgb
from datetime import datetime

/kaggle/input/15th-data60sec/var_list.csv
/kaggle/input/15th-data60sec/train_label.csv
/kaggle/input/15th-data60sec/test_data.pickle/test_data.pickle
/kaggle/input/15th-data60sec/train_data.pickle/train_data.pickle


In [2]:
train = pd.read_pickle('/kaggle/input/15th-data60sec/train_data.pickle/train_data.pickle')
test = pd.read_pickle('/kaggle/input/15th-data60sec/test_data.pickle/test_data.pickle')
train_label = train['label']

var_list = pd.read_csv('/kaggle/input/15th-data60sec/var_list.csv')
var_list = ['time'] + list(var_list['time'])

In [3]:
var_list = list(reversed(sorted(list(set(var_list) - set(train.columns[train.isna().any()].tolist() + test.columns[test.isna().any()].tolist())))))

train = train.loc[(train['time'] >= 10),var_list].reset_index(drop=True)
test = test.loc[(test['time'] >= 10),list(set(var_list) - set(['label']))].reset_index(drop=True)

In [4]:
print(train[var_list].shape)
print(test[list(set(var_list) - set(['label']))].shape)

(41400, 2580)
(36000, 2579)


In [5]:
train = train.groupby('id').mean().reset_index()
train_label = train['label']
train_id = train['id']

test = test.groupby('id').mean().reset_index()
test_id = test['id']

var_model = list(set(train.columns) & set(test.columns) - set(['id']))

In [6]:
def tr_vl_split(train_df, num, seed):    
    np.random.seed(seed)
    
    valid_id = []
    temp = list(train_df['label'].value_counts()[train_df['label'].value_counts() > num].index)
    for a in temp:
        id_list = list(train_df[train_df['label'] == a]['id'])
        valid_id += random.sample(id_list,num)
    
    train_id = list(set(train_df['id']) - set(valid_id))
    
    x_tr_ = train[train['id'].isin(train_id)]
    y_tr_ = train_label[train['id'].isin(train_id)]

    x_vl_ = train[~train['id'].isin(train_id)]
    y_vl_ = train_label[~train['id'].isin(train_id)]
    
    return x_tr_, y_tr_, x_vl_, y_vl_ 

In [7]:
x_tr, y_tr, x_vl, y_vl = tr_vl_split(train, 3, seed = 1995)

In [8]:
lgb_tr = lgb.Dataset(x_tr[var_model], label=y_tr)
lgb_vl = lgb.Dataset(x_vl[var_model], label=y_vl)
lgb_test = test[var_model]

watchlist_1 = [lgb_tr, lgb_vl]
watchlist_2 = [lgb_vl, lgb_tr]

params = {
    "objective": "multiclass",
    "boosting": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 2,
    "metric": "multiclass",
    "num_class" : 198
}

lgb_model = lgb.train(params, train_set=lgb_tr, num_boost_round=500, valid_sets=watchlist_1, verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.707729	valid_1's multi_logloss: 2.56919
[200]	training's multi_logloss: 0.170748	valid_1's multi_logloss: 2.30285
[300]	training's multi_logloss: 0.0682004	valid_1's multi_logloss: 2.2389
[400]	training's multi_logloss: 0.0389761	valid_1's multi_logloss: 2.21287
[500]	training's multi_logloss: 0.0266024	valid_1's multi_logloss: 2.19722
Did not meet early stopping. Best iteration is:
[500]	training's multi_logloss: 0.0266024	valid_1's multi_logloss: 2.19722


In [9]:
prediction = pd.DataFrame(lgb_model.predict(test[var_model]))

In [10]:
time_now = datetime.now()
submission_name = str(time_now)[:16] + '_submission.csv'
pd.concat([pd.DataFrame(test_id),prediction],axis=1).to_csv(submission_name,index=False)